In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import distributions as dist
from torch.utils.data import DataLoader, TensorDataset
from torch import optim

from torchvision.datasets import MNIST, FashionMNIST
from torchvision import transforms as tr
from torchvision.models import vgg11_bn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint

from pathlib import Path
import pickle
from torch import nn


In [ ]:
from inpainting.datasets.mnist import train_val_datasets
from inpainting.visualizations.digits import digit_with_mask as vis_digit_mask
from inpainting.training import train_inpainter
from inpainting.utils import classifier_experiment, inpainted, predictions_for_entire_loader
import inpainting.visualizations.samples as vis
from inpainting.datasets import mask_coding as mc
from inpainting.datasets.utils import RandomRectangleMaskConfig
from inpainting.custom_layers import Reshape
from inpainting.losses import nll_masked_batch_loss, nll_masked_batch_loss_components, r2_masked_batch_loss, nll_masked_batch_loss_same_size_masks
from inpainting.inpainters.mnist import MNISTLinearInpainter, MNISTConvolutionalInpainter
from inpainting.inpainters.fullconv import FullyConvolutionalInpainter
from inpainting import backbones as bkb
from inpainting import losses2 as l2

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = "white"

In [ ]:
!ps aux | grep mprzewie

In [ ]:
!echo $CUDA_VISIBLE_DEVICES
!nvidia-smi

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
device

In [ ]:
experiment_path = Path("../results/mnist/fullconv/28x28/32_2_32_no_latent_a_ampl_0.5_bl_2")
experiment_path.mkdir(exist_ok=True, parents=True)

In [ ]:
ds_train, ds_val = train_val_datasets(
    Path.home() / "uj/data/",
    mask_configs=[
#         RandomRectangleMaskConfig(mc.UNKNOWN_LOSS, 28,28, 0,0),

        RandomRectangleMaskConfig(mc.UNKNOWN_LOSS, 14, 14, 0,0),
#         RandomRectangleMaskConfig(mc.UNKNOWN_NO_LOSS, 16,16, 0,0)
    ],
    ds_type=MNIST,
    resize_size=(28,28),
)

fig, axes = plt.subplots(10, 10, figsize=(15, 15))
for i in range(100):
    (x,j), y = ds_train[i]
    ax = axes[i // 10, i%10]
    ax.set_title(f"{y}")
    vis_digit_mask(x, j,ax)
train_fig = plt.gcf()
train_fig.savefig(experiment_path / "train.png")
plt.show()

In [ ]:
batch_size=196
dl_train = DataLoader(ds_train, batch_size, shuffle=True)
dl_val = DataLoader(ds_val, batch_size, shuffle=False)

In [ ]:
log_noms = lambda x,j,p,m,a,d : nll_masked_batch_loss_components(
    x,j,p,m,a,d
)["log_noms"]

x_minus_means = lambda x,j,p,m,a,d : nll_masked_batch_loss_components(
    x,j,p,m,a,d
)["x_minus_means"]

mse = lambda x,j,p,m,a,d : nll_masked_batch_loss_components(
    x,j,p,m,a,d
)["x_minus_means_2"]

log_dets = lambda x,j,p,m,a,d : nll_masked_batch_loss_components(
    x,j,p,m,a,d
)["log_dets"]

log_2_pi = lambda x,j,p,m,a,d : nll_masked_batch_loss_components(
    x,j,p,m,a,d
)["log_2_pi"]

a_var = lambda x,j,p,m,a,d : a.var()
a_mean = lambda x,j,p,m,a,d : a.mean()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
# device = torch.device("cpu")
# inpainter = MNISTLinearInpainter(n_mixes=1, hidden_size=2048)


inpainter = FullyConvolutionalInpainter(
    n_mixes=1, a_width=4, a_amplitude=0.5, c_h_w=(1, 28, 28), last_channels=32, 
    extractor=bkb.down_up_backbone(
        (2, 28, 28),
        depth=2,
        first_channels=32, 
        last_channels=32,
        kernel_size=5,
        latent=False,
        block_length=2,   
    )
#     extractor=bkb.UNet(
#         input_nc=2,
#         output_nc=32,
#         ngf=32
#     )
)

# inpainter = MNISTConvolutionalInpainter(a_amplitude=0.5, h_w=(32,32), last_channels=64)

history = []


opt = optim.Adam(inpainter.parameters(), lr=4e-5)

In [ ]:
history = train_inpainter(
    inpainter, 
    dl_train, 
    dl_val, 
    opt, 
    loss_fn =l2.nll_buffered, 
    n_epochs=200,
    device=device,
    history_start=history,
    # tqdm_loader=True,
    max_benchmark_batches=50,
    losses_to_log=dict(
        mse=l2.loss_factory(
            gathering_fn=l2.buffered_gather_batch_by_mask_indices,
            calc_fn=l2.mse
        ),
        nll=l2.nll_buffered, 
    )
)

In [ ]:
len(history)

In [ ]:
# [
#     h["losses"]
#     for h in history
# ]

In [ ]:

with (experiment_path / "history_last_epoch.pkl").open("wb") as f:
    history_tmp = history
    pickle.dump(history[-1], f)

In [ ]:
with (experiment_path / "inpainter.schema").open("w") as f:
    print(inpainter, file=f)
with (experiment_path / "opt.schema").open("w") as f:
    print(opt, file=f)

torch.save(inpainter.state_dict(), experiment_path / "inpainter.state")

In [ ]:
chckp = torch.load(experiment_path / "inpainter.state")
chckp
inpainter.load_state_dict(chckp)

In [ ]:
len(history)

In [ ]:
history[-1]["losses"]["objective"]

In [ ]:
for loss_name in set(history[0]["losses"].keys()):
    for fold in ["train", "val"]:
        
        plt.plot(
            list(range(len(history))),
            [h["losses"][loss_name][fold] for h in history],
            label=fold
        )
    plt.title(loss_name)
    plt.legend()
    fig = plt.gcf()
    fig.savefig(experiment_path / f"history.{loss_name}.png")
    plt.show()

In [ ]:
skip = 5

row_length = vis.row_length(*list(zip(*history[0]["sample_results"]["train"]))[0])
fig, axes = plt.subplots(
    int(np.ceil(len(history) / skip)* 2), 
    row_length,
    figsize=(20, 30)
)


for e, h in enumerate(history):
    if e % skip !=0 and e != (len(history) -1):
        continue
    
    for ax_no, fold in [(0,"train"), (1,"val")]:
        x, j, p, m, a, d, y = [t[0] for t in  h["sample_results"][fold]]
        row_no = (e // skip)*2 + ax_no
        print(m.mean(), m.min(), m.max())

        vis.visualize_sample(
            x, j, p, m, a, d, y, 
            ax_row=axes[row_no], 
            title_prefixes={
                0: f"{e} {fold} ",
#                 1: f"y_m = {y_masked_pred}"
            },
            drawing_fn=vis_digit_mask
        )

epochs_fig = plt.gcf()
epochs_fig.savefig(experiment_path / "epochs_renders.png")

In [ ]:
epochs_path = experiment_path / "epochs"
epochs_path.mkdir(exist_ok=True)
skip = 5
n_rows = 16

for e, h in enumerate(history):
    if e % skip !=0 and e != (len(history) -1):
        continue
    
    for ax_no, fold in [(0,"train"), (1,"val")]:
        
        
        row_length = vis.row_length(*list(zip(*h["sample_results"][fold]))[0])

        fig, axes = plt.subplots(
            n_rows, 
            row_length,
            figsize=(20, 30)
        )

        for row_no, (x, j, p, m ,a, d, y) in enumerate(list(zip(*h["sample_results"][fold]))[:n_rows]):
            vis.visualize_sample(
                x, j, p, m, a, d, y, 
                ax_row=axes[row_no], 
                title_prefixes={
                    0: f"{e} {fold} ",
#                     1: f"y_m = {y_masked_pred}"
                },
                drawing_fn=vis_digit_mask

            )
        
        title = f"{e}_{fold}"
        plt.suptitle(title)
        plt.savefig(epochs_path / f"{title}.png")
#       

In [ ]:
x, j, p, m, a, d, y= history[0]["sample_results"]["train"]
plt.imshow(x[0][0])

m[0][0]

In [ ]:
import pickle

with (experiment_path / "history_last_epoch.pkl").open("rb") as f:
    hist_last_epoch = pickle.load(f)

# history = [hist_last_epoch]

In [ ]:
import matplotlib.pyplot as plt

def plot_arrays_stats(
    arrays, ax=None, stat_fns = [np.min, np.max, np.mean], markers="."
):
    if ax is None:
        fig, ax = plt.subplots()
        
    if isinstance(markers, str):
        markers = [markers] * len(stat_fns)
    
    for fn, m in zip(stat_fns, markers):
        ax.scatter(
            range(len(arrays)),
            [
                fn(a) for a in arrays
            ],
            marker=m,
            label=fn.__name__
            
        )
    
    return ax

In [ ]:
for fold in ["val"]:
    x, j, p, m, a, d, y = hist_last_epoch["sample_results"][fold]

    a_resh = a.reshape(a.shape[0] * a.shape[1], a.shape[2], a.shape[3])
    covs = a_resh.transpose(0, 2, 1) @ a_resh
    
    fig, ax = plt.subplots(figsize=(20, 25), nrows=5)
    
    ax[0].set_title(f"m stats {fold}")
    plot_arrays_stats(
        m, ax[0]
    )
        
    samples = [
        vis.gans_gmms_sample_no_d(x_, m_[0], a_[0], d_[0])
        for (x_, m_, a_, d_) in zip(x, m ,a, d)
    ]
    
    ax[1].set_title("samples stats")
    plot_arrays_stats(samples, ax[1])
    
    ax[2].set_title(f"a stats {fold}")
    plot_arrays_stats(
        a, ax[2]
    )
    
    ax[3].set_title(f"d stats {fold}")
    plot_arrays_stats(
        d, ax[3]
    )
    
    ax[4].set_title(f"cov stats {fold}")
    plot_arrays_stats(
        covs, ax[4]
    )
    [a.legend() for a in ax[:5]]
    fig.savefig(experiment_path / f"outputs_stats.png")
    plt.show()
    
    cov_resh = covs[0].reshape(-1)
    plt.hist(cov_resh, log=True, bins=100)
    plt.title(f"cov[0] hist {fold}")
    plt.show()
    
    cov = covs[0]
    
    fig, ax = plt.subplots(figsize=(10, 10), nrows=2)
    eigs = np.linalg.eigvals(cov)
    ax[0].scatter(range(len(eigs)), eigs)
    ax[0].set_title("eigenvals of cov[0]")

    cov_d = cov + np.diag(d[0])
    eigs_d = np.linalg.eigvals(cov_d)
    ax[1].scatter(range(len(eigs_d)), eigs_d)
    ax[1].set_title("eigenvals of cov[0] + d[0]")
    fig.savefig(experiment_path / "eigenvals.png")
    plt.show()

    # wygląda na to, że mamy ~3 duże wartosci własne
    
    print("m analysis")
    
    
    plt.hist(d[0].reshape(-1), bins=100, log=True)
    plt.title("d[0] hist")
    plt.show()
    
#     for i in range(3):
#         plt.imshow(a_resh[0, i].reshape(28,28), cmap="gray")
#         plt.show()
    
    
    
    

In [ ]:
# next up
# * różne aktywacje wyjścia macierzy A (sigmoid? różne zakresy?)
# * wyplotować wielkość wartości własne A, D na przestrzeni treningu
# spróbować trenowania na niepełnych danych

In [ ]:
val_results = predictions_for_entire_loader(
    inpainter.to(torch.device("cpu")),
    dl_val,
    torch.device("cpu")
    
)
with (experiment_path / "val_predictions.pkl").open("wb") as f:
    pickle.dump(val_results, f)
    

In [ ]:
with (experiment_path / "val_predictions.pkl").open("wb") as f:
    pickle.dump(val_results, f)
    

In [ ]:
fig, ax = plt.subplots(10, 12, figsize=(15,15)
)

for i in range(10):
    vis.visualize_sample(
        *val_results[i],
        drawing_fn=vis_digit_mask,
        title_prefixes=dict(),
        ax_row=ax[i]
    )